In [1]:
from abc import ABC, abstractmethod
import numpy as np

###############################
# Base Potential Class (Abstract)
###############################

class PotentialEnergySurface(ABC):
    """Abstract base class for potential energy surfaces."""

    def __init__(self): 
        self.max_acceptable_force_mag = np.inf # will be updated by ABC later
        self.energy_calls = 0 
        self.force_calls = 0
    
    @abstractmethod
    def _potential(self, position: np.ndarray) -> float: 
        """Compute potential energy at given position."""
        pass

    def potential(self, position: np.ndarray) -> float: 
        """
        Wrapper for potential calls. 
        
        DO NOT EDIT: Users should implement _potential()
        """
        self.energy_calls += 1
        return self._potential(position)
        
    @abstractmethod
    def default_starting_position(self) -> np.ndarray:
        """Return default starting position for this PES."""
        pass
    
    def _gradient(self, position) -> np.ndarray:
        """Compute analytic gradient at given position, if available
        Raise NotImplementedError if not implemented.
        
        If your potential has no analytical gradient, simply omit this method 
        from your implementation, and the ABC will perform finite-difference.
        """
        raise NotImplementedError("Analytic gradient not implemented for this PES.")
    
    def gradient(self, position) -> np.ndarray:
        """
        Wrapper for _gradient with built-in force magnitude limiting
        DO NOT EDIT: Users should implement _gradient() for analytical gradient calculation
        """
        if (norm := np.linalg.norm(grad := self._gradient(position))) > self.max_acceptable_force_mag: 
            print(f"Warning: Gradient value of {grad} detected as likely unphysically large in magnitude; shrunk to magnitude {self.max_acceptable_force_mag}")
            grad = self.max_acceptable_force_mag * grad / norm
        
        self.force_calls += 1
        return grad       

    def plot_range(self) -> tuple:
        """Return plotting range for visualization."""
        return None
        
    def known_minima(self) -> list[np.ndarray]:
        """Return known basins (for analysis)."""
        return None 

    def known_saddles(self) -> list[np.ndarray]:
        """Return known saddles (for analysis)."""
        return None


class Complex1D(PotentialEnergySurface):
    
    def _potential(self, x):
        a=[6.5, 4.2, -7.3, -125]
        b=[2.5, 4.3, 1.5, 0.036]
        c=[9.7, 1.9, -2.5, 12]
        V = x**2
        for i in range(4):
            exponent = -b[i]*(x-c[i])**2
            exponent = np.clip(exponent, -100, 100)
            V += a[i]*np.exp(exponent)
        return V
     
    def default_starting_position(self):
        return np.array([0.0], dtype=float)
        
    def plot_range(self):
        return (-3.5, 11.6)
        
    def known_minima(self):
        return [
                np.array([-2.27151]), 
                np.array([0.41295]), 
                np.array([2.71638]), 
                np.array([8.69999]), 
                np.array([10.35518]) 
                ]
    
    def known_saddles(self):
        return [
                np.array([-1.2645]),
                np.array([1.94219]), 
                np.array([4.55508]),
                np.array([9.7913])
                ]

In [ ]:
# Build it back up one thing at a time and make sure everything works as you go 
potential = Complex1D()
potential.potential(1.2)



np.float64(0.07448190647585928)